### **MBAN 5110 - Midterm Exam**

### **PART 1**

##### Import Relevant Libraries

In [5]:
import numpy as np
import pandas as pd
from statsmodels.sandbox.regression.gmm import GMM

##### Upload the Dataset

In [77]:
df_part1 = pd.read_csv("https://raw.githubusercontent.com/GoldenSnow-Xue/schulich_data_science/main/MBAN%205110/Midterm%20Project/midterm_partone.csv")

In [78]:
df_part1

,Constant,Stock Change,Inventory Turnover,Operating Profit,Interaction Effect,Current Ratio,Quick Ratio,Debt Asset Ratio
0,1,0.870332,1.795946,0.115846,0.208053,1.672527,0.255171,0.473317
1,1,-0.047347,1.395501,0.436967,0.609788,1.637261,0.221763,0.489967
2,1,0.001176,1.664563,0.541016,0.900555,1.640619,0.189141,0.374269
3,1,-0.901200,1.605738,0.539399,0.866133,1.436221,0.131944,0.224399
4,1,-0.176353,1.591451,0.539938,0.859285,1.433140,0.183095,0.213446
...,...,...,...,...,...,...,...,...
1691,1,-0.015543,5.225766,0.309119,1.615384,3.554503,2.197871,0.005549
1692,1,0.399089,5.324390,0.274782,1.463044,3.745006,2.324502,0.004359
1693,1,-0.702200,5.575258,0.287503,1.602905,3.434909,2.282626,0.000000
1694,1,0.283926,5.423463,0.256657,1.391968,2.876645,1.454948,0.000000


##### 1. Update the GMM model

In [79]:
y_vals = np.array(df_part1["Stock Change"])
x_vals = np.array(df_part1[["Inventory Turnover", "Operating Profit", "Interaction Effect"]])
iv_vals = np.array(df_part1[["Current Ratio", "Quick Ratio", "Debt Asset Ratio"]])

class gmm(GMM):
    def momcond(self, params):
        p0, p1, p2, p3, delta = params
        endog = self.endog
        exog = self.exog
        inst = self.instrument

        error0 = endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2]
        error1 = (endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2]) * exog[:,1]
        error2 = (endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2]) * exog[:,2]
        error3 = (endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2]) * inst[:,0] - delta
        error4 = (endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2]) * inst[:,1] - delta
        error5 = (endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2]) * inst[:,2] - delta

        g = np.column_stack((error0, error1, error2, error3, error4, error5))
        return g
    
beta0 = np.array([0.1, 0.1, 0.1, 0.1, 0.1])
res = gmm(endog = y_vals, exog = x_vals, instrument = iv_vals, k_moms=6, k_params=5).fit(beta0)

res.summary()


Optimization terminated successfully.
         Current function value: 0.000031
         Iterations: 10
         Function evaluations: 15
         Gradient evaluations: 15
Optimization terminated successfully.
         Current function value: 0.000345
         Iterations: 9
         Function evaluations: 11
         Gradient evaluations: 11
Optimization terminated successfully.
         Current function value: 0.000346
         Iterations: 7
         Function evaluations: 10
         Gradient evaluations: 10
Optimization terminated successfully.
         Current function value: 0.000346
         Iterations: 2
         Function evaluations: 5
         Gradient evaluations: 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                                 gmm Results                                  
==============================================================================
Dep. Variable:                      y   Hansen J:                       0.5862
Model:                            gmm   Prob (Hansen J):                 0.444
Method:                           GMM                                         
Date:                Sun, 12 Nov 2023                                         
Time:                        14:57:20                                         
No. Observations:                1696                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
p 0           -0.0208      0.021     -0.986      0.324      -0.062       0.020
p 1            0.0011      0.001      1.839      0.066   -7.31e-05       0.002
p 2           -0.1062      0.032     -3.316      0.001      -0.169      -0.043
p 3            0.0011      0.000      2.688      0.007       0.000       0.002
p 4           -0.0006      0.003     -0.213      0.831      -0.006       0.005
==============================================================================
"""

##### 2. Analyze the GMM summary table

Null Hypothesis: The industry expert claims that there is a bias in moment conditions of instrumental variables.

**p4 (Coefficient of δ)**

P-Value: 0.831

P-Value is greater than 0.05, reject the null hypothesis, the industry expert's claim isn't statistically justified.

(Details in the summary report)


### **PART 2**

##### 1. Divide the dataset equally into two as training (50%) and test (50%) sets. Use the training set to fit a logistic regression model, where the credit rating is the dependent variable. Apply the model to the test set, and report the confusion matrix, recall, precision, and F1 score values.

##### **Data Preparation**

##### Import Relevant Libraries

In [40]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, recall_score, precision_score, f1_score

##### Upload the Dataset

In [9]:
df_part2 = pd.read_csv("https://raw.githubusercontent.com/GoldenSnow-Xue/schulich_data_science/main/MBAN%205110/Midterm%20Project/midterm_parttwo.csv")

In [10]:
df_part2

,Years of Education after High School,Requested Credit Amount,Number of Dependents,Monthly Income,Monthly Expense,Marital Status,Credit Rating
0,1,Low,No dependent,Very low,Very low,Married,Positive
1,2,Low,No dependent,Very low,Very low,Single,Positive
2,1,Low,No dependent,Very low,Very low,Single,Positive
3,3,Low,No dependent,Very low,Very low,Married,Positive
4,3,Low,No dependent,Very low,Very low,Single,Negative
...,...,...,...,...,...,...,...
8076,3,Low,Less than 2,Very High,Very high,Married,Positive
8077,3,Medium,Less than 2,Very High,Very high,Married,Negative
8078,3,Medium,More than 2,Very High,Very high,Married,Positive
8079,7,Medium,Less than 2,Very High,Very high,Married,Positive


In [12]:
df_part2.describe()

,Years of Education after High School
count,8081.000000
mean,2.608588
std,1.571835
min,0.000000
25%,1.000000
50%,3.000000
75%,3.000000
max,7.000000


In [11]:
df_part2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8081 entries, 0 to 8080
Data columns (total 7 columns):
 #   Column                                Non-Null Count  Dtype 
---  ------                                --------------  ----- 
 0   Years of Education after High School  8081 non-null   int64 
 1   Requested Credit Amount               8081 non-null   object
 2   Number of Dependents                  8081 non-null   object
 3   Monthly Income                        8081 non-null   object
 4   Monthly Expense                       8081 non-null   object
 5   Marital Status                        8081 non-null   object
 6   Credit Rating                         8081 non-null   object
dtypes: int64(1), object(6)
memory usage: 442.1+ KB


In [14]:
X = df_part2.drop('Credit Rating', axis=1)
y = df_part2['Credit Rating']

In [16]:
# Identify numerical and categorical columns
num_cols = X.select_dtypes(include=['float64', 'int64']).columns
cat_cols = X.select_dtypes(include=['object']).columns

In [25]:
# Create a column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_cols),
        ('cat', OneHotEncoder(), cat_cols)
    ]
)

##### **Data Splitting**

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)

##### **Model Fitting**

In [36]:
# Create a pipeline
pipeline = Pipeline([
                    ('preprocessor', preprocessor),
                    ('classifier', LogisticRegression())
])


In [37]:
# Fit the model
pipeline.fit(X_train, y_train)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  Index(['Years of Education after High School'], dtype='object')),
                                                 ('cat', OneHotEncoder(),
                                                  Index(['Requested Credit Amount', 'Number of Dependents', 'Monthly Income',
       'Monthly Expense', 'Marital Status'],
      dtype='object'))])),
                ('classifier', LogisticRegression())])

##### **Model Prediction and Evaluation**

In [39]:
# Make predictions
y_pred = pipeline.predict(X_test)

In [42]:
# Compute metrics
conf_matrix = confusion_matrix(y_test, y_pred)
recall = recall_score(y_test, y_pred, pos_label='Positive')
precision = precision_score(y_test, y_pred, pos_label='Positive')
f1 = f1_score(y_test, y_pred, pos_label='Positive')

In [56]:
# Displaying the results
print("Confusion Matrix:\n", conf_matrix)
print("Recall:", recall)
print("Precision:", precision)
print("F1 Score:", f1)

Confusion Matrix:
 [[   0  567]
 [   0 3474]]
Recall: 1.0
Precision: 0.8596881959910914
F1 Score: 0.9245508982035928


**Confusion Matrix:**
True Negative (TN): 0
False Positive (FP): 567
False Negative (FN): 0
True Positive (TP): 3474

**Recall:**
The recall score is 1.0. 

**Precision :**
The precision score is approximately 0.860. 

**F1-Score:**
The F1-Score, which balances precision and recall, is approximately 0.925.

(Details in the summary report)

##### 2. Suppose that the bank decided to make the credit approval process more challenging such that only 15% of the applications would be granted. Calculate the threshold value for the prediction probability, so only 15% of the test set would get their applications approved.

In [73]:
# Getting the predicted probabilities for the positive class
y_pred_proba = pipeline.predict_proba(X_test)[:, 1]

# Calculating the threshold for top 15% scores
threshold = np.percentile(y_pred_proba, 85)

# Applying the new threshold to make predictions
y_pred_new = (y_pred_proba >= threshold)

In [74]:
# Ensure y_pred_new is in the same format as y_test
# Convert the boolean predictions to string labels ('Negative', 'Positive')
y_pred_new_labels = np.where(y_pred_new, 'Positive', 'Negative')


In [75]:
# Recalculating metrics with the new threshold
conf_matrix_new = confusion_matrix(y_test, y_pred_new_labels)
recall_new = recall_score(y_test, y_pred_new_labels, pos_label='Positive')
precision_new = precision_score(y_test, y_pred_new_labels, pos_label='Positive')
f1_new = f1_score(y_test, y_pred_new_labels, pos_label='Positive')

In [76]:
# Displaying the results
print("New Confusion Matrix:\n", conf_matrix_new)
print("New Recall:", recall_new)
print("New Precision:", precision_new)
print("New F1 Score:", f1_new)

New Confusion Matrix:
 [[ 485   82]
 [2901  573]]
New Recall: 0.16493955094991364
New Precision: 0.8748091603053435
New F1 Score: 0.27754904335190117


**New Confusion Matrix:**
True Negative (TN): 485
False Positive (FP): 82
False Negative (FN): 2901
True Positive (TP): 573

**New Recall:** 
The recall score is approximately 0.165. 

**New Precision:** 
The precision score is approximately 0.875. 

**New F1 Score:** The F1 score is approximately 0.278. 

(Details in the summary report)
